### Loading bitcoin prices

In [1]:
import requests as rq
from datetime import datetime as dt
import pandas as pd
import numpy as np

In [22]:
API_LINK = "https://api.coindesk.com/v1/bpi/historical/close.json"
start = "2010-07-17"
end = dt.now().strftime("%Y-%m-%d")
URL = API_LINK + "?start=" + start + "&end=" + end

API_Data = rq.get(API_LINK)

PARAMS = {"start":start,
          "end": end} 
r = rq.get(url = API_LINK, params = PARAMS) 
data = r.json()["bpi"]


dates = np.array(list(data.keys()))
dates = np.vectorize(lambda x: dt.strptime(x, "%Y-%m-%d"))(dates)
prices = np.array(list(data.values()))

bitcoin_price = pd.DataFrame({"Date": dates, "ClosePrice": prices})

periods = 14

bitcoin_price["Change"] = bitcoin_price["ClosePrice"].pct_change(periods=periods) * 100.0
bitcoin_price = bitcoin_price.fillna(0)
bitcoin_price

,Date,ClosePrice,Change
0,2010-07-18,0.0858,0.000000
1,2010-07-19,0.0808,0.000000
2,2010-07-20,0.0747,0.000000
3,2010-07-21,0.0792,0.000000
4,2010-07-22,0.0505,0.000000
...,...,...,...
3586,2020-05-12,8729.9500,13.021498
3587,2020-05-13,9099.2000,9.980721
3588,2020-05-14,9582.5825,9.631124
3589,2020-05-15,9523.5950,8.573406


In [23]:
bitcoin_price_dates = np.array(bitcoin_price["Date"].values)
bitcoin_price_values = np.array(bitcoin_price["ClosePrice"].values)

### Preparacion datos

In [162]:
history_size = 20 #days
target_size = 5 #days

In [163]:
def get_data_serie(values, history_size, target_size):
    
    start_pos = 0
    lavel_pos = history_size + target_size
    
    data = []
    lavel = []
    std = []
    mean = []
    
    while lavel_pos < len(values):
        current_data = values[start_pos:start_pos + history_size]
        current_std = np.std(current_data)
        current_mean = np.mean(current_data)
        current_data = (current_data - current_mean) / current_std
        current_lavel = (values[lavel_pos] - current_mean) / current_std
        
        data.append(current_data)
        lavel.append(current_lavel)
        mean.append(current_mean)
        std.append(current_std)
        
        start_pos += 1
        lavel_pos += 1
        
    return np.array(data), np.array(lavel), np.array(mean), np.array(std)


In [164]:
data, lavel, mean, std = get_data_serie(bitcoin_price_values, history_size, target_size)

### Recurrent Neural Network

In [43]:
import tensorflow as tf

In [136]:
BATCH_SIZE = 256
BUFFER_SIZE = 10000

left_out = 0.1
train_size = 0.7 

In [165]:
np.random.seed(1234)

train_test_size = int(len(data) * (1 - left_out))

train_ids = np.random.choice(range(train_test_size),
                             size=int(len(data) * train_size),
                             replace=False)

In [166]:
train_mask = np.zeros(train_test_size, dtype=bool)
train_mask[train_ids] = True

test_mask = np.ones(train_test_size, dtype=bool)
test_mask[train_ids] = False

In [167]:
train_data = data[:train_test_size][train_mask]
train_lavel = lavel[:train_test_size][train_mask]
test_data = data[:train_test_size][test_mask]
test_lavel = lavel[:train_test_size][test_mask]

In [168]:
train_data_x = np.array([ [[y] for y in x] for x in train_data])
test_data_x = np.array([ [[y] for y in x] for x in test_data])

train_univariate = tf.data.Dataset.from_tensor_slices((train_data_x, train_lavel))
train_univariate = train_univariate.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

test_univariate = tf.data.Dataset.from_tensor_slices((test_data_x, test_lavel))
test_univariate = test_univariate.batch(BATCH_SIZE).repeat()

In [169]:
simple_lstm_model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(8, input_shape=train_data_x.shape[-2:]),
    tf.keras.layers.Dense(1)
])

simple_lstm_model.compile(optimizer='adam', loss='mae')

simple_lstm_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 8)                 320       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 9         
Total params: 329
Trainable params: 329
Non-trainable params: 0
_________________________________________________________________


In [170]:
EVALUATION_INTERVAL = 200
EPOCHS = 50

simple_lstm_model.fit(train_univariate, epochs=EPOCHS,
                      steps_per_epoch=EVALUATION_INTERVAL,
                      validation_data=test_univariate, validation_steps=50)

Epoch 1/50
200/200 [==============================] - 2s 8ms/step - loss: 2.0077 - val_loss: 1.6299
Epoch 2/50
200/200 [==============================] - 1s 7ms/step - loss: 1.6760 - val_loss: 1.5027
Epoch 3/50
200/200 [==============================] - 1s 7ms/step - loss: 1.5918 - val_loss: 1.4552
Epoch 4/50
200/200 [==============================] - 1s 7ms/step - loss: 1.5637 - val_loss: 1.4372
Epoch 5/50
200/200 [==============================] - 1s 7ms/step - loss: 1.5511 - val_loss: 1.4290
Epoch 6/50
200/200 [==============================] - 1s 7ms/step - loss: 1.5431 - val_loss: 1.4250
Epoch 7/50
200/200 [==============================] - 1s 7ms/step - loss: 1.5380 - val_loss: 1.4215
Epoch 8/50
200/200 [==============================] - 1s 7ms/step - loss: 1.5335 - val_loss: 1.4191
Epoch 9/50
200/200 [==============================] - 1s 7ms/step - loss: 1.5301 - val_loss: 1.4183
Epoch 10/50
200/200 [==============================] - 1s 7ms/step - loss: 1.5279 - val_loss: 1.4181

In [171]:
predictions = []
iterations = 0

for x in data:
    x_input = x.reshape((1, history_size, 1))
    result = simple_lstm_model.predict(x_input)
    predictions.append((result[0][0] * std[iterations]) + mean[iterations])
    
    iterations += 1
    if iterations % 250 == 0:
        print(iterations)
        
predictions = np.array(predictions)

250
500
750
1000
1250
1500
1750
2000
2250
2500
2750
3000
3250
3500


### Result test

In [177]:
from bokeh.layouts import gridplot
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import LinearAxis, Range1d
from bokeh.models import ColumnDataSource, RangeTool, HoverTool
from bokeh.layouts import row, column
from bokeh.models.callbacks import CustomJS

In [181]:
p = figure(tools="pan,wheel_zoom,box_zoom,reset,save",
           title="Predicciones Bitcoin",
           x_axis_label="Time", y_axis_label="USD",
           plot_width=1600, plot_height=800,
           x_axis_type='datetime')

p.add_tools(HoverTool(tooltips=[('Fecha','@x{%F}'),
                                ('Valor','@y')],
                      formatters={'@x':'datetime'},
                      mode='vline'))


true_values = (lavel * std) + mean

p.line(bitcoin_price_dates[history_size + target_size:],
       true_values,
       color='lightskyblue',
       legend_label='Precio real')

p.line(bitcoin_price_dates[history_size + target_size:],
       mean,
       color='deeppink',
       legend_label='Prediccion con promedio')

p.line(bitcoin_price_dates[history_size + target_size:],
       predictions,
       color='seagreen',
       legend_label='RNN')

output_notebook()
show(p)

Loading BokehJS ...

In [182]:
p = figure(tools="pan,wheel_zoom,box_zoom,reset,save",
           title="Error predicciones Bitcoin",
           x_axis_label="Time", y_axis_label="USD",
           plot_width=1600, plot_height=800,
           x_axis_type='datetime')


p.add_tools(HoverTool(tooltips=[('Fecha','@x{%F}'),
                                ('Valor','@y')],
                      formatters={'@x':'datetime'},
                      mode='vline'))

error_media = np.absolute((mean - true_values) / true_values)
error_rnn = np.absolute((predictions - true_values) / true_values)

p.line(bitcoin_price_dates[history_size + target_size:],
       error_media,
       color='deeppink',
       legend_label='Error media')

p.line(bitcoin_price_dates[history_size + target_size:],
       error_rnn,
       color='seagreen',
       legend_label='Error media')

output_notebook()
show(p)

Loading BokehJS ...

In [174]:
print("Error promedio media: " + str(np.mean(error_media) * 100.0))
print("Error promedio rnn: " + str(np.mean(error_rnn) * 100.0))

Error promedio media: 13.25140831939712
Error promedio rnn: 8.349677075746856
